In [1]:
%matplotlib inline

import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(style="darkgrid")

In [2]:
db = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "C04eL5@rn",
    database = "soil-sensor-data"
)

mycursor = db.cursor()
print(db)

In [3]:
mycursor.execute("SELECT * FROM sensor01_test")

output = mycursor.fetchall()

for x in output:
    print(x)

('1741393550.169714', 29.1, 362)
('1741393552.5227919', 29.0, 361)
('1741393554.8369532', 29.1, 361)
('1741393556.3387918', 29.3, 361)


In [4]:
query = "SELECT * FROM sensor01_test"

s1 = pd.read_sql(query, db)

C:\Users\Dell\AppData\Local\Temp\ipykernel_28540\2708161193.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s1 = pd.read_sql(query, db)


In [5]:
s1.head()

,Time in Millisecond Since UTC Epoch,Temperature in Celsius,Capacitive Reading
0,1741393550.169714,29.1,362
1,1741393552.5227919,29.0,361
2,1741393554.8369532,29.1,361
3,1741393556.3387918,29.3,361


In [6]:
df2 = pd.read_csv('plantDatabase.csv')

df2.columns = ['#', 'Name', 'Common Name', 'Type', 'New Jersey Native?', 'Soil Type', 'Soil Moisture', 'Soil pH', 'Drought Tolerance', 'Optimal Light', 'Light Range', 'Hardiness Zone']

df3 = df2.drop(df2.index[0])

df4 = df3.drop(columns=['#'])

dbfinal = df4

In [7]:
conditions = [
    (s1['Capacitive Reading'] <= 650),
    (s1['Capacitive Reading'] > 650) & (s1['Capacitive Reading'] <= 1100),
    (s1['Capacitive Reading'] > 1100) & (s1['Capacitive Reading'] <= 1550),
    (s1['Capacitive Reading'] > 1550)
]

values = ['Dry', 'Dry_Moist', 'Moist', 'Moist_Wet']

s1['Soil Moisture'] = np.select(conditions, values, default=pd.NaT)

s1.head()

,Time in Millisecond Since UTC Epoch,Temperature in Celsius,Capacitive Reading,Soil Moisture
0,1741393550.169714,29.1,362,Dry
1,1741393552.5227919,29.0,361,Dry
2,1741393554.8369532,29.1,361,Dry
3,1741393556.3387918,29.3,361,Dry


In [8]:
s1.dtypes

Time in Millisecond Since UTC Epoch     object
Temperature in Celsius                 float64
Capacitive Reading                       int64
Soil Moisture                           object
dtype: object

In [9]:
#extract an array of plants that have the specified soil moisture from the plant database, 
#and store the array into the respective lists

dryplants = dbfinal.loc[dbfinal['Soil Moisture'] == 'Dry', 'Common Name']
print(dryplants)

drymoist = dbfinal.loc[dbfinal['Soil Moisture'] == 'Dry_Moist', 'Common Name']
print(drymoist)

moistplants = dbfinal.loc[dbfinal['Soil Moisture'] == 'Moist', 'Common Name']
print(moistplants)

moistwet = dbfinal.loc[dbfinal['Soil Moisture'] == 'Moist_Wet', 'Common Name']
print(moistwet)

2                  Yarrow
8       Black Huckleberry
10     Lindheimer’s Muhly
11        Little Bluestem
13    Japanese Tree Lilac
15             Arborvitae
17      Lowbush Blueberry
20                 Zinnia
Name: Common Name, dtype: object
1     Narrowleaf Mountain mint
3               Yellow Buckeye
4                      Begonia
9             Spotted Geranium
16           American Basswood
18           New York Ironweed
19                   Arrowwood
Name: Common Name, dtype: object
14    King of the Meadow
Name: Common Name, dtype: object
5     Ornamental Cabbage
6       Crested Woodfern
7             Sweetbells
12             Woolgrass
Name: Common Name, dtype: object


In [10]:
con2 = [
    (s1['Soil Moisture'] == 'Dry'),
    (s1['Soil Moisture'] == 'Dry_Moist'),
    (s1['Soil Moisture'] == 'Moist'),
    (s1['Soil Moisture'] == 'Moist_Wet')
]

#convert the lists of plants with a specified soil moisture into strings so they  
#can be returned as recommendations in a new column of the soil moisture dataframe

dpst = " ".join(str(x) for x in dryplants)
dmst = " ".join(str(x) for x in drymoist)
mpst = " ".join(str(x) for x in moistplants)
mwst = " ".join(str(x) for x in moistwet)

val2 = [dpst, dmst, mpst, mwst]

s1['Plant Recommendations'] = np.select(con2, val2, default=pd.NaT)
s1.head()

,Time in Millisecond Since UTC Epoch,Temperature in Celsius,Capacitive Reading,Soil Moisture,Plant Recommendations
0,1741393550.169714,29.1,362,Dry,Yarrow Black Huckleberry Lindheimer’s Muhly Li...
1,1741393552.5227919,29.0,361,Dry,Yarrow Black Huckleberry Lindheimer’s Muhly Li...
2,1741393554.8369532,29.1,361,Dry,Yarrow Black Huckleberry Lindheimer’s Muhly Li...
3,1741393556.3387918,29.3,361,Dry,Yarrow Black Huckleberry Lindheimer’s Muhly Li...


In [11]:
for i in s1['Plant Recommendations']:
    if i == dpst:
        print(dryplants)
    elif i == dmst:
        print (drymoist)
    elif i == mpst:
        print (moistplants)
    elif i == mwst:
        print (moistwet)

2                  Yarrow
8       Black Huckleberry
10     Lindheimer’s Muhly
11        Little Bluestem
13    Japanese Tree Lilac
15             Arborvitae
17      Lowbush Blueberry
20                 Zinnia
Name: Common Name, dtype: object
2                  Yarrow
8       Black Huckleberry
10     Lindheimer’s Muhly
11        Little Bluestem
13    Japanese Tree Lilac
15             Arborvitae
17      Lowbush Blueberry
20                 Zinnia
Name: Common Name, dtype: object
2                  Yarrow
8       Black Huckleberry
10     Lindheimer’s Muhly
11        Little Bluestem
13    Japanese Tree Lilac
15             Arborvitae
17      Lowbush Blueberry
20                 Zinnia
Name: Common Name, dtype: object
2                  Yarrow
8       Black Huckleberry
10     Lindheimer’s Muhly
11        Little Bluestem
13    Japanese Tree Lilac
15             Arborvitae
17      Lowbush Blueberry
20                 Zinnia
Name: Common Name, dtype: object
